In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import random
import matplotlib.pyplot as plt
import math
from Utils import preprocess, string_to_tensor, plot_graphs
from AkshrantarDataset import AksharantarDataset

# Importing data

### Selecting Language

In [2]:
language = 'tam'
start_token = '<'
end_token = '>'
pad_token = ' '
unk_token = '~'

In [ ]:
data = AksharantarDataset(language)
criterion = nn.CrossEntropyLoss(ignore_index=data.tr_l2i[pad_token])

target_dict_count = len(data.tr_l2i)
english_dict_count = len(data.en_l2i)

In [ ]:
# path = 'aksharantar_sampled/' + language

# train_df = pd.read_csv(path+'/'+language+'_train.csv', header=None)
# test_df = pd.read_csv(path+'/'+language+'_test.csv', header=None)
# val_df = pd.read_csv(path+'/'+language+'_valid.csv', header=None)

# train_source, train_target = train_df[0].tolist(), train_df[1].tolist()
# test_source, test_target = test_df[0].tolist(), test_df[1].tolist()
# val_source, val_target = val_df[0].tolist(), val_df[1].tolist()

### Printing Sample

num_sample = 5
for i in range(num_sample):
    print(f'{train_source[i]}  -->  {train_target[i]}')

In [5]:
# english_chars = list(set(''.join(train_source) + start_token + end_token + pad_token + unk_token))
# target_chars = list(set(''.join(train_target) + start_token + end_token + pad_token + unk_token))

# english_dict_count = len(english_chars)
# target_dict_count = len(target_chars)

# print("Number of Charecters : ", len(data.tr_l2i))

# print("Target Charecters: ")
# for i, c in enumerate(target_chars):
#     print(i, c, end='\t')
#     if i % 5 == 0:
#         print()
# print()

Number of Charecters :  50
Target Charecters: 
0 ய	
1 ற	2 <	3 ே	4 ச	5 உ	
6 ோ	7 ை	8 ு	9 த	10 ந	
11 ஸ	12 ்	13 ஒ	14 ண	15 ங	
16 ஞ	17 ஈ	18 ஊ	19 வ	20 ொ	
21 ஐ	22 ௌ	23 ள	24 ழ	25 ஷ	
26 ட	27 ப	28 ஃ	29 >	30 எ	
31 ஜ	32 க	33 ீ	34 ல	35 ூ	
36 இ	37 ம	38 ஆ	39 ன	40 அ	
41 ஏ	42 ி	43 ா	44 ஹ	45 ெ	
46 ர	47 ஓ	48 ~	49  	


In [6]:
# en_i2l, en_l2i = {}, {}
# tr_i2l, tr_l2i = {}, {}

# for i, x in enumerate(english_chars):
#     en_l2i[x] = i
#     en_i2l[i] = x

# for i, x in enumerate(target_chars):
#     tr_l2i[x] = i
#     tr_i2l[i] = x

# Utility Functions


def preprocess(strings, start_token, end_token, pad_token):
    """Adds start and end token and adds padding"""
    res = []
    max_len = len(max(strings, key=len))

    for item in strings:
        temp = start_token + item + end_token
        temp = temp.ljust(max_len+2, pad_token) #2 is added, because we added start and end token to each word
        res.append(temp)

    return res


def string_to_tensor(strings, l2i_dict, unk_token):
    """
    replaces the chareceters of the sting with corrospong ix (by refering l2i_dict) and returns as int tensor
    """
    res = torch.zeros(len(strings), len(strings[0]))
    
    for i in range(len(strings)):
        for j in range(len(strings[i])):
            if strings[i][j] not in l2i_dict :
                res[i][j] = l2i_dict[unk_token]
            else:
                res[i][j] = l2i_dict[strings[i][j]]
        
    return res.type(torch.LongTensor)


def plot_graphs(training_errors, validation_errors, training_accuracy, validation_accuracy):
    """
    Plots a Error and Accuracy graphs for training and validation data over the epochs

    Params:
    -----
    training_errors: list containing error (ie loss values) of the training data over the epochs
    validation_errors: list containing error (ie loss values) of the validation data over the epochs
    training_accuracy: list containing accuracy of the training data over the epochs
    validation_accuracy: list containing accuracy of the validation data over the epochs

    Returns:
    -----
    fig: matplot figure object 
    """
    x = np.arange(len(training_errors))
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(x, training_errors, label = "Training Error")
    ax1.plot(x, validation_errors, label = "Validation Error")
    ax1.set_title("Errors")
    ax1.legend(fontsize= 'small', loc='upper right')
    ax2.plot(x, training_accuracy, label = "Training Accuracy")
    ax2.plot(x, validation_accuracy, label = "Validation Accuracy")
    ax2.set_title("Accuracy")
    ax2.legend(fontsize = 'small',loc='upper right')
    return fig

### Preprocessing test and validation data

In [8]:
# val_source_tensor = string_to_tensor(preprocess(val_source, start_token, end_token, pad_token), en_l2i, unk_token).transpose(0,1).requires_grad_(False)
# val_target_tensor = string_to_tensor(preprocess(val_target, start_token, end_token, pad_token), tr_l2i, unk_token).transpose(0,1).requires_grad_(False)

# test_source_tensor = string_to_tensor(preprocess(test_source, start_token, end_token, pad_token), en_l2i, unk_token).transpose(0,1).requires_grad_(False)
# test_target_tensor = string_to_tensor(preprocess(test_target, start_token, end_token, pad_token), tr_l2i, unk_token).transpose(0,1).requires_grad_(False)

# Encoder

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers = 1, p = 0, bi_dir = False, rnn_class = nn.GRU):
        """
        Init Parameters:
        input_size : english_dict_count
        embedding_size : size of each embedding vector
        hidden_size : size of hidden state vector
        num_layers : number of recurrent layers of RNN
        p : dropout probability
        rnn_class: type of RNN to be used in the encoder

        Input:
        x : torch.Tensor of shape (seq_length, N)
            where seq_length - len of longest string in the batch
            N - batch size
        
        Outpus:
        outputs: torch.Tensor of shape (seq_len, N, hidden_size * D), where D = 2 if bi_dir = True else 1
        hidden: torch.Tensor of shape (num_layers * D, N, hidden_size)
        
        cell: torch.Tensor of shape (num_layers * D, N, hidden_size) if(rnn_class == "LSTM")
        """
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn_class= rnn_class
        self.rnn = rnn_class(embedding_size, hidden_size, num_layers, dropout=p, bidirectional = bi_dir)


    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        if(self.rnn_class.__name__ == "LSTM"):
            outputs, (hidden, cell) = self.rnn(embedding)
            # outputs shape: (seq_length, N, hidden_size)
        else:
            outputs, hidden = self.rnn(embedding)
        
        if(self.rnn_class.__name__ == "LSTM"):
            return outputs, hidden, cell
        else:
            return outputs, hidden


# Decoder

In [10]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1, p = 0, bi_dir = False, rnn_class = nn.GRU):
        """input size = output size = target language charecters
        Init Parameters:
        input_size: target_dict_count
        embedding_size: size of each embedding vector
        hidden_size: size of hidden state vector
        output_size: number of output features in fully connected layer
        num_layers : number of recurrent layers of RNN
        p : dropout probability
        rnn_class: type of RNN to be used in the encoder

        Input:
        x: torch.Tensor of shape (N)
        hidden: torch.Tensor of shape (num_layers * D, N, hidden_size)
        cell: torch.Tensor of shape (num_layers * D, N, hidden_size)

        Outputs:
        predications: torch.Tensor of shape (N, target_dict_count), where D = 2 if bi_dir = True else 1
        hidden: torch.Tensor of shape (num_layers * D, N, hidden_size)
        
        cell: torch.Tensor of shape (num_layers * D, N, hidden_size) if(rnn_class == "LSTM")
        """
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn_class = rnn_class
        self.rnn = rnn_class(embedding_size, hidden_size, num_layers, dropout=p, bidirectional = bi_dir)

        self.D = 1
        if(bi_dir == True):
            self.D = 2
        self.fc = nn.Linear(hidden_size * self.D, output_size)


    def forward(self, x, hidden, cell = None):
        #cell is set to none, for GRU and RNN

        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        # print(x.shape, hidden.shape, cell.shape)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)
        
        if(self.rnn_class.__name__ == "LSTM"):
            outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
            # outputs shape: (1, N, hidden_size * D)
        else:
            outputs, hidden = self.rnn(embedding, hidden)
            

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        if(self.rnn_class.__name__ == "LSTM"):
            return predictions, hidden, cell
        else:
            return predictions, hidden

# Model

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        encoder_layers = encoder.num_layers
        decoder_layers = decoder.num_layers
        D = decoder.D #we set bidiretion as common in both encoder and decoder, so no need to check for D value seperately
        self.enc_to_dec = nn.Linear(encoder_layers*D, decoder_layers*D)
        self.rnn_class = decoder.rnn_class #we use same rnn in both encoder and decoder

    def forward(self, source, target, teacher_force_ratio=0):
        """source : (source_len, N) - not sure
        teacher_forching_ratio : probability in which original values is favored over predicted values
                                if 0 : predicted values is passed for all chars in target
                                if 1 : true values is passed for all chars in target

        """
        batch_size = source.shape[1] 
        target_len = target.shape[0]
        target_vocab_size = target_dict_count

        # print("source shape ", source.shape)
        # print("target shape ", target.shape)
        # print("N : ", batch_size)
        # print("tar len : ", target_len)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        # print("outputs shape : ", outputs.shape)

        
        if(self.rnn_class.__name__ == "LSTM"):
            _, hidden, cell = self.encoder(source)
        else:
            _, hidden = self.encoder(source)

        N = hidden.shape[1]
        hidden_size= hidden.shape[2]
        # hidden, cell shape: (D*encoder_layers, N, hidden_size)

        hidden = hidden.transpose(0, 2) # hidden shape: (hidden_size, N, D*encoder_layers)
        hidden = hidden.reshape(-1, hidden.shape[2]) # hidden shape: (hidden_size * N, D*encoder_layers)
        hidden = self.enc_to_dec(hidden) # hidden shape: (hidden_size * N, D*decoder_layers)
        hidden = hidden.reshape(hidden_size, N, hidden.shape[1]) # hidden shape: (hidden_size, N, D*decoder_layers)
        hidden = hidden.transpose(0,2) # hidden shape: (D*decoder_layers, N, hidden_size)

        if(self.rnn_class.__name__ == "LSTM"):
            #at all the below steps, cell will have the shape of hidden
            cell = cell.transpose(0,2)
            cell = cell.reshape(-1, cell.shape[2])
            cell = self.enc_to_dec(cell)
            cell = cell.reshape(hidden_size, N, cell.shape[1])
            cell = cell.transpose(0,2)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]
        outputs[:, :, data.tr_l2i[start_token]] = 1 #setting prob = 1 for starting token 

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            if(self.rnn_class.__name__ == "LSTM"):
                output, hidden, cell = self.decoder(x, hidden, cell)
            else:
                output, hidden = self.decoder(x, hidden)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess
        # print("OUTPUTS: ", outputs)
        return outputs


    def calc_accuracy(self, output, target):
        """
        output: torch.Tensor of shape (seq_len, N)
        target: torch.Tensor of shape (seq_len, N)
        """
        # batch_size = 32
        running_acc = 0
        seq_len = output.shape[0]
        N = output.shape[1]
        matched_strings = 0
        with torch.no_grad():
            for j in range(N):
                current_word_matched = True
                for i in range(seq_len):
                    if(target[i][j] in {data.tr_l2i[pad_token]}): #we dont care whatever prediction in the pad_token place
                        continue
                    if(output[i][j] != target[i][j]): #compare the predictions of charecters, start and end_token places
                        current_word_matched = False
                        break
                if(current_word_matched == True):
                    matched_strings += 1
        return matched_strings*100 / N 


    def calc_evaluation_metrics(self, soruce_strings, target_strings):
        """

        Returns:
        loss: loss value for the current batch of strings
        accuracy: accuracy value for the current batch of strings
        """
        batch_size = 32
        loss  = 0
        with torch.no_grad():
            no_of_batch = 0
            running_accuracy = 0
            for i in range(0, len(soruce_strings), batch_size):
                inp_data = string_to_tensor(preprocess(soruce_strings[i:i+batch_size], start_token, end_token, pad_token), data.en_l2i, unk_token).transpose(0,1)
                target = string_to_tensor(preprocess(target_strings[i:i+batch_size], start_token, end_token, pad_token), data.tr_l2i, unk_token).transpose(0,1)

                output = self(inp_data, target)
                running_accuracy += self.calc_accuracy(output.argmax(2), target)

                output = output.reshape(-1, output.shape[2])
                target = target.reshape(-1)

                loss = criterion(output, target)
                loss += loss.item()
                no_of_batch += 1
        return loss, running_accuracy/no_of_batch
    
 

# Training

In [20]:
# Training hyperparameters
num_epochs = 30
learning_rate = 0.01
batch_size = 32

# Model hyperparameters
load_model = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder = english_dict_count
input_size_decoder = target_dict_count
output_size = target_dict_count
embedding_size = 32
encoder_layers = 2
decoder_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5
hidden_size = 64
bi_directional = True
rnn = nn.LSTM

In [21]:
enc = Encoder(english_dict_count, embedding_size, hidden_size, 
              num_layers=encoder_layers, 
              bi_dir=bi_directional,
              p=enc_dropout,
              rnn_class=rnn)
dec = Decoder(target_dict_count, embedding_size, hidden_size, target_dict_count, 
              num_layers=decoder_layers, 
              bi_dir=bi_directional, 
              p = dec_dropout,
              rnn_class=rnn)

mod = Seq2Seq(enc, dec)

optimizer = optim.Adam(mod.parameters(), lr=learning_rate)

In [22]:
def temp_print(output):
    """output shape: target_seq_length * N"""
    res = []
    for j in range(output.shape[1]):
        temp = ""
        for i in range(output.shape[0]):
            temp += tr_i2l[output[i,j].item()]
        
        res.append(temp)
    return res

In [23]:
loss_list, acc_list, val_loss_list, val_acc_list = [], [], [], []

init_teacher_forcing_ratio = 0.8
for epoch in range(num_epochs):
    running_loss = 0
    cur_batch = 0
    running_accuracy = 0
    total_batches = math.ceil(len(train_source)/batch_size)
    for i in range(0, len(train_source), batch_size):
        src_strings = train_source[i: i+batch_size]
        tar_strings = train_target[i: i+batch_size]

        src_strings = preprocess(src_strings, start_token, end_token, pad_token)
        tar_strings = preprocess(tar_strings, start_token, end_token, pad_token)

        #transposing to make the shape as expected
        inp_data = string_to_tensor(src_strings, en_l2i, unk_token).transpose(0,1)
        target = string_to_tensor(tar_strings, tr_l2i, unk_token).transpose(0,1)

        #here teacher forcing ratio will reduces linearly from init_teacher_forcing_ratio to 0 in half the number of epochs
        teacher_forcing_ratio = max(0, init_teacher_forcing_ratio * (1 - (epoch*2/num_epochs)))
        output = mod(inp_data, target, teacher_forcing_ratio)

        running_accuracy += mod.calc_accuracy(output.argmax(2), target)

        # print("train: output : ", output.shape)
        if(epoch in {math.floor(0.8 * num_epochs), num_epochs-1}):
            result = temp_print(output.argmax(2))
            print("result: ", result)
            print("target: ", tar_strings)
        
        # print("output: ",result)
        # print("target:", tar_strings)

        # print("op before reshopsed ", output.shape)
        # print("tar before reshape: ", target.shape)

        output = output.reshape(-1, output.shape[2])
        target = target.reshape(-1)

        # print("op: ", output.shape)
        # print("tar: ", target.shape)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()
        torch.nn.utils.clip_grad_norm_(mod.parameters(), max_norm=1)
        optimizer.step()
        running_loss += loss.item()
        cur_batch += 1        
        
        #comment this line when running with a log file (or in kaggle)
        print(f"Epoch {epoch+1:3d} | Batches: {cur_batch} / {total_batches}", end='\r')

    # val_output = mod(val_source_tensor, val_target_tensor)
    # val_output = val_output.reshape(-1, val_output.shape[2])
    # val_target_tensor = val_target_tensor.reshape(-1)
    # # print(f"val output : {val_output.shape} \t val_target: {val_target_tensor.shape}")
    # val_loss = criterion(val_output, val_target_tensor)

    val_loss, val_accuracy= mod.calc_evaluation_metrics(val_source, val_target)
    # val_loss = 0


    print(f"[Epoch {epoch+1:3d}/{num_epochs}] \t Loss: {(running_loss/total_batches):.3f}\t Acc: {(running_accuracy/total_batches):2.2f} \t Val Loss: {val_loss:2.3f}\t Val Acc: {val_accuracy:2.2f}")
    loss_list.append(running_loss/total_batches)
    acc_list.append(running_accuracy/total_batches)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_accuracy)

fig = plot_graphs(loss_list, val_loss_list, acc_list, val_acc_list)

[Epoch   1/30] 	 Loss: 1.411	 Acc: 0.01 	 Val Loss: 3.695	 Val Acc: 0.12
[Epoch   2/30] 	 Loss: 0.936	 Acc: 0.09 	 Val Loss: 2.878	 Val Acc: 0.24
[Epoch   3/30] 	 Loss: 0.843	 Acc: 0.19 	 Val Loss: 2.932	 Val Acc: 0.46
[Epoch   4/30] 	 Loss: 0.805	 Acc: 0.23 	 Val Loss: 2.293	 Val Acc: 0.46
[Epoch   5/30] 	 Loss: 0.790	 Acc: 0.32 	 Val Loss: 2.235	 Val Acc: 0.68
[Epoch   6/30] 	 Loss: 0.783	 Acc: 0.44 	 Val Loss: 2.209	 Val Acc: 0.59
[Epoch   7/30] 	 Loss: 0.784	 Acc: 0.42 	 Val Loss: 2.423	 Val Acc: 0.61
[Epoch   8/30] 	 Loss: 0.795	 Acc: 0.42 	 Val Loss: 2.111	 Val Acc: 0.49
[Epoch   9/30] 	 Loss: 0.801	 Acc: 0.48 	 Val Loss: 1.951	 Val Acc: 0.34
[Epoch  10/30] 	 Loss: 0.808	 Acc: 0.46 	 Val Loss: 1.857	 Val Acc: 0.49


KeyboardInterrupt: 

In [24]:
i = 100
batch_size = 20

src_strings = train_source[i: i+batch_size]
tar_strings = train_target[i: i+batch_size]

src_strings = preprocess(src_strings, start_token, end_token, pad_token)
tar_strings = preprocess(tar_strings, start_token, end_token, pad_token)

#transposing to make the shape as expected
inp_data = string_to_tensor(src_strings, en_l2i, unk_token).transpose(0,1)
target = string_to_tensor(tar_strings, tr_l2i, unk_token).transpose(0,1)

#here teacher forcing ratio will reduces linearly from init_teacher_forcing_ratio to 0 in half the number of epochs
teacher_forcing_ratio = max(0, init_teacher_forcing_ratio * (1 - (epoch*2/num_epochs)))
output = mod(inp_data, target, teacher_forcing_ratio)

result = temp_print(output.argmax(2))
print("result: ", result)
print("target: ", tar_strings)

result:  ['<வசிப்படத்திிலும்>>', '<இசைக்கின்றனர்>>>>>', '<ரஷிதாவும்>>>்>>>>>', '<அக்கானிகளை>>>>>>>>', '<கூப்பெட்டாங்க>>>>>', '<வருசத்திலிருந்து>்', '<ஸ்விரே>>>>>>>>>>>்', '<சிப்போராலோடும்>>>>', '<நாடகீயம்>>>>>>>>>>', '<வழித்த>>>>>>>>>>>>', '<கயானியிடம்>>>>>>>்', '<கடுஷு>>>>>>>>>>>>>', '<மலைக்கூட்டத்தில்>>', '<மிருகளிதையை>>>>>>>', '<மாபெரும்வேற்றியை>>', '<திருவிளனரகளள்>>>>>', '<தெரியவன்ததல்த>>>>>', '<வங்கிகளாலும்>>>>>>', '<நம்னாவிடில்>>>>>>>', '<நட்புபார்ப்புவார்>']
target:  ['<வசிப்பிடத்திலும்> ', '<இசைக்கின்றனர்>    ', '<ரஷிதாவும்>        ', '<அக்காணிகளை>       ', '<கூப்பிட்டாங்க>    ', '<வருசத்திலிருந்து> ', '<ஸ்வைர்>           ', '<சிப்போராளோடும்>   ', '<நாடகீயம்>         ', '<வஹித்த>           ', '<கயானியிடம்>       ', '<கடுசு>            ', '<மலைக்கூட்டத்தில்> ', '<மிருகவதையை>       ', '<மாபெரும்வெற்றியை> ', '<திருவிளநகரில்>    ', '<தெரியவந்தால்தான்> ', '<வங்கிகளாலும்>     ', '<நம்பாவிடில்>      ', '<நட்புபாராட்டுபவர்>']
